# Ice thickness inversion

This example shows how to run the OGGM ice thickness inversion model
with various ice parameters: the deformation parameter A and a sliding
parameter (fs).

There is currently no "best" set of parameters for the ice thickness
inversion model. As shown in
[Maussion et al. (2019)](https://www.geosci-model-dev.net/12/909/2019/),
the default parameter set results in global volume estimates which are a bit
larger than previous values. For the consensus estimate of
[Farinotti et al. (2019)](https://www.nature.com/articles/s41561-019-0300-3),
OGGM participated with a deformation parameter A that is 1.5 times larger than the
generally accepted default value.

There is no reason to think that the ice parameters are the same between
neighboring glaciers. There is currently no "good" way to calibrate them,
or at least no generaly accepted one.
We won't discuss the details here, but we provide a script to illustrate
the sensitivity of the model to this choice.

**New in version 1.4**: we demonstrate how to apply a new global task in OGGM, `workflow.calibrate_inversion_from_consensus` to calibrate the A parameter to match the consensus estimate from [Farinotti et al. (2019)](https://www.nature.com/articles/s41561-019-0300-3).

At the end of this tutorial, we show how to distribute the "flowline thicknesses" on a glacier map.

## Run

In [ ]:
# Libs
import geopandas as gpd

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks, graphics

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')
rgi_region = '11'  # Region Central Europe

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Inversion')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

# This is useful here
cfg.PARAMS['use_multiprocessing'] = True

# RGI file
path = utils.get_rgi_region_file(rgi_region)
rgidf = gpd.read_file(path)

# Select the glaciers in the Pyrenees
rgidf = rgidf.loc[rgidf['O2Region'] == '2']

# Sort for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)

# Go - get the pre-processed glacier directories
# We start at level 3, because we need all data for the inversion
gdirs = workflow.init_glacier_directories(rgidf, from_prepro_level=3, prepro_border=10)

# Because of recent changes in OGGM not yet available in the preprocessed directories, 
# we re-run this task:
workflow.execute_entity_task(tasks.prepare_for_inversion, gdirs)

# Default parameters
# Deformation: from Cuffey and Patterson 2010
glen_a = 2.4e-24
# Sliding: from Oerlemans 1997
fs = 5.7e-20

In [ ]:
with utils.DisableLogger():  # this scraps some output - to use with caution!!!
    
    # Correction factors
    factors = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    factors += [1.1, 1.2, 1.3, 1.5, 1.7, 2, 2.5, 3, 4, 5]
    factors += [6, 7, 8, 9, 10]

    # Run the inversions tasks with the given factors
    for f in factors:
        # Without sliding
        suf = '_{:03d}_without_fs'.format(int(f * 10))
        workflow.execute_entity_task(tasks.mass_conservation_inversion, gdirs,
                                     glen_a=glen_a*f, fs=0)
        # Store the results of the inversion only
        utils.compile_glacier_statistics(gdirs, filesuffix=suf,
                                         inversion_only=True)

        # With sliding
        suf = '_{:03d}_with_fs'.format(int(f * 10))
        workflow.execute_entity_task(tasks.mass_conservation_inversion, gdirs,
                                     glen_a=glen_a*f, fs=fs)
        # Store the results of the inversion only
        utils.compile_glacier_statistics(gdirs, filesuffix=suf,
                                         inversion_only=True)

## Read the data

The data are stored as csv files in the working directory. The easiest way to read them is to use [pandas](http://pandas.pydata.org/)!

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import os

Let's read the output of the inversion with the default OGGM parameters first:

In [ ]:
df = pd.read_csv(os.path.join(WORKING_DIR, 'glacier_statistics_011_without_fs.csv'), index_col=0)
df

There are only 35 glaciers in the Pyrenees! That's why the run was relatively fast.

## Results

One way to visualize the output is to plot the volume as a function of area in a log-log plot, illustrating the well known volume-area relationship of mountain glaciers:

In [ ]:
ax = df.plot(kind='scatter', x='rgi_area_km2', y='inv_volume_km3')
ax.semilogx(); ax.semilogy()
xlim, ylim = [1e-2, 0.7], [1e-5, 0.05]
ax.set_xlim(xlim); ax.set_ylim(ylim);

As we can see, there is a clear relationship, but it is not perfect. Let's fit a line to these data (the "volume-area scaling law"):

In [ ]:
# Fit in log space 
dfl = np.log(df[['inv_volume_km3', 'rgi_area_km2']])
slope, intercept, r_value, p_value, std_err = stats.linregress(dfl.rgi_area_km2.values, dfl.inv_volume_km3.values)

In their seminal paper, [Bahr et al. (1997)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/97JB01696) describe this relationship as:

$$V = \alpha S^{\gamma}$$

With V the volume in km$^3$, S the area in km$^2$ and $\alpha$ and $\gamma$ the scaling parameters (0.034 and 1.375, respectively). How does OGGM compare to these in the Pyrenees?

In [ ]:
print('power: {:.3f}'.format(slope))
print('slope: {:.3f}'.format(np.exp(intercept)))

In [ ]:
ax = df.plot(kind='scatter', x='rgi_area_km2', y='inv_volume_km3', label='OGGM glaciers')
ax.plot(xlim, np.exp(intercept) * (xlim ** slope), color='C3', label='Fitted line')
ax.semilogx(); ax.semilogy()
ax.set_xlim(xlim); ax.set_ylim(ylim);
ax.legend();

## Sensitivity analysis 

Now, let's read the output files of each run separately, and compute the regional volume out of them:

In [ ]:
dftot = pd.DataFrame(index=factors)
for f in factors:
    # Without sliding
    suf = '_{:03d}_without_fs'.format(int(f * 10))
    fpath = os.path.join(WORKING_DIR, 'glacier_statistics{}.csv'.format(suf))
    _df = pd.read_csv(fpath, index_col=0, low_memory=False)
    dftot.loc[f, 'without_sliding'] = _df.inv_volume_km3.sum()
    
    # With sliding
    suf = '_{:03d}_with_fs'.format(int(f * 10))
    fpath = os.path.join(WORKING_DIR, 'glacier_statistics{}.csv'.format(suf))
    _df = pd.read_csv(fpath, index_col=0, low_memory=False)
    dftot.loc[f, 'with_sliding'] = _df.inv_volume_km3.sum()

And plot them:

In [ ]:
dftot.plot();
plt.xlabel('Factor of Glen A (default 1)'); plt.ylabel('Regional volume (km$^3$)');

As you can see, there is quite a difference between the solutions. In particular, close to the default value for Glen A, the regional estimates are very sensitive to small changes in A. The calibration of A is a problem that has yet to be resolved by global glacier models...

## New in version 1.4: calibrate to match the consensus estimate 

Here, a "best Glen A" is found in order that the total inverted volume of the glaciers of gdirs fits to the 2019 consensus estimate. 

In [ ]:
cdf = workflow.calibrate_inversion_from_consensus(gdirs, filter_inversion_output=False)

In [ ]:
cdf.sum()

Note that here we calibrate the Glen A parameter to a value that is equal for all glaciers of gdirs (here $A \sim 9.504\cdot A_0$), i.e. we calibrate to match the total volume of all glaciers and not to match them individually.

In [ ]:
cdf.iloc[:3]

*just as a side note, "vol_bsl_itmix_m3" means volume below sea level and is therefore zero for these Alpine glaciers!*

## Distributed ice thickness 

The OGGM inversion and dynamical models use the "1D" flowline assumption: for some applications, you might want to use OGGM to create distributed ice thickness maps. Currently, OGGM implements two ways to "distribute" the flowline thicknesses, but only [the simplest one](https://docs.oggm.org/en/stable/generated/oggm.tasks.distribute_thickness_per_altitude.html) works robustly:

In [ ]:
# Distribute
workflow.execute_entity_task(tasks.distribute_thickness_per_altitude, gdirs);

We just created a new output of the model, which we can access in the `gridded_data` file: 

In [ ]:
# xarray is an awesome library! Did you know about it?
import xarray as xr
ds = xr.open_dataset(gdirs[0].get_filepath('gridded_data'))

In [ ]:
ds.distributed_thickness.plot();

## Plot many glaciers on a map 

Let's select a group of glaciers close to each other: 

In [ ]:
rgi_ids = ['RGI60-11.0{}'.format(i) for i in range(3205, 3211)]
sel_gdirs = [gdir for gdir in gdirs if gdir.rgi_id in rgi_ids]
graphics.plot_googlemap(sel_gdirs)
# you might need to install motionless if it is not yet in your environment

### Using OGGM 

Since [a recent PR (21.05.2020)](https://github.com/OGGM/oggm/pull/1025), OGGM can plot the thickness of a group of glaciers on a map:

In [ ]:
graphics.plot_distributed_thickness(sel_gdirs)

This is however not very useful because OGGM can only plot on a map as large as the local glacier map of the first glacier in the list. See [this issue](https://github.com/OGGM/oggm/issues/1007) for a discussion about why.

### Using salem 

Under the hood, OGGM uses [salem](https://salem.readthedocs.io) to make the plots. Let's do that for our case: it requires some manual tweaking, but it should be possible to automatize this better in the future.

*Note: this also requires a very recent version of salem to work (21.05.2020)*

In [ ]:
import salem
# Make a grid covering the desired map extent
g = salem.mercator_grid(center_ll=(0.65, 42.64), extent=(4000, 4000))
# Create a map out of it
smap = salem.Map(g, countries=False)
# Add the glaciers outlines
for gdir in sel_gdirs:
    crs = gdir.grid.center_grid
    geom = gdir.read_pickle('geometries')
    poly_pix = geom['polygon_pix']
    smap.set_geometry(poly_pix, crs=crs, fc='none', zorder=2, linewidth=.2)
    for l in poly_pix.interiors:
        smap.set_geometry(l, crs=crs, color='black', linewidth=0.5)
f, ax = plt.subplots(figsize=(6, 6))
smap.visualize();

In [ ]:
# Now add the thickness data
for gdir in sel_gdirs:
    grids_file = gdir.get_filepath('gridded_data')
    with utils.ncDataset(grids_file) as nc:
        vn = 'distributed_thickness'
        thick = nc.variables[vn][:]
        mask = nc.variables['glacier_mask'][:]
    thick = np.where(mask, thick, np.NaN)
    # The "overplot=True" is key here
    # this needs a recent version of salem to run properly
    smap.set_data(thick, crs=gdir.grid.center_grid, overplot=True)

In [ ]:
# Set colorscale and other things
smap.set_cmap(graphics.OGGM_CMAPS['glacier_thickness'])
smap.set_plot_params(nlevels=256)
# Plot
f, ax = plt.subplots(figsize=(6, 6))
smap.visualize(ax=ax, cbar_title='Glacier thickness (m)');

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)